<a href="https://colab.research.google.com/github/nmuonko/Bagging-and-bosting-project/blob/main/Bagging_Boosting_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [214]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import zscore

In [215]:
data = pd.read_csv('/content/talking_data.csv')

In [216]:
data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [217]:
data.shape

(100000, 8)

In [218]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   ip               100000 non-null  int64 
 1   app              100000 non-null  int64 
 2   device           100000 non-null  int64 
 3   os               100000 non-null  int64 
 4   channel          100000 non-null  int64 
 5   click_time       100000 non-null  object
 6   attributed_time  227 non-null     object
 7   is_attributed    100000 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 6.1+ MB


In [219]:
data.isnull().values.any()

True

In [220]:
data.isnull().sum()

ip                     0
app                    0
device                 0
os                     0
channel                0
click_time             0
attributed_time    99773
is_attributed          0
dtype: int64

In [221]:
data.nunique()

ip                 34857
app                  161
device               100
os                   130
channel              161
click_time         80350
attributed_time      227
is_attributed          2
dtype: int64

CLick Time has the maximum  number of unique values

In [222]:
data.drop("attributed_time",axis=1,inplace=True)

In [223]:
data.isnull().values.any()

False

In [224]:
app_click = data.groupby('app').size().reset_index(name='count')
app_click = app_click[app_click['count']>app_click['count'].quantile(.8)]
app_click.head()

,app,count
0,1,3135
1,2,11737
2,3,18279
4,5,188
5,6,1303


In [225]:
data['click_time'] = pd.to_datetime(data['click_time'])

In [226]:
data['hour']=data['click_time'].dt.hour
data['day_of_week']=data['click_time'].dt.day_of_week
data['day_of_year']=data['click_time'].dt.day_of_year
data['month']=data['click_time'].dt.month

In [227]:
data.head()

,ip,app,device,os,channel,click_time,is_attributed,hour,day_of_week,day_of_year,month
0,87540,12,1,13,497,2017-11-07 09:30:38,0,9,1,311,11
1,105560,25,1,17,259,2017-11-07 13:40:27,0,13,1,311,11
2,101424,12,1,19,212,2017-11-07 18:05:24,0,18,1,311,11
3,94584,13,1,13,477,2017-11-07 04:58:08,0,4,1,311,11
4,68413,12,1,1,178,2017-11-09 09:00:09,0,9,3,313,11


In [228]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ip             100000 non-null  int64         
 1   app            100000 non-null  int64         
 2   device         100000 non-null  int64         
 3   os             100000 non-null  int64         
 4   channel        100000 non-null  int64         
 5   click_time     100000 non-null  datetime64[ns]
 6   is_attributed  100000 non-null  int64         
 7   hour           100000 non-null  int64         
 8   day_of_week    100000 non-null  int64         
 9   day_of_year    100000 non-null  int64         
 10  month          100000 non-null  int64         
dtypes: datetime64[ns](1), int64(10)
memory usage: 8.4 MB


In [229]:
data.drop("click_time", axis=1, inplace=True)

In [230]:
data.head()

,ip,app,device,os,channel,is_attributed,hour,day_of_week,day_of_year,month
0,87540,12,1,13,497,0,9,1,311,11
1,105560,25,1,17,259,0,13,1,311,11
2,101424,12,1,19,212,0,18,1,311,11
3,94584,13,1,13,477,0,4,1,311,11
4,68413,12,1,1,178,0,9,3,313,11


In [231]:
from sklearn.model_selection import train_test_split # # Import Sklearn package's data splitting function
from sklearn import metrics
from sklearn import preprocessing

X = data.drop(['is_attributed'], axis=1)
y = data['is_attributed']

# Split X and y into training and test set in 80:20 ratio

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [232]:
y.value_counts()

0    99773
1      227
Name: is_attributed, dtype: int64

In [233]:
ZX = preprocessing.scale(X)
ZX_train, ZX_test, Zy_train, Zy_test = train_test_split(ZX, y, test_size=0.2, random_state=10)

In [234]:
Zy_train.value_counts()

0    79812
1      188
Name: is_attributed, dtype: int64

In [235]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=34)
ZX_train,Zy_train = sm.fit_resample(ZX_train,Zy_train)

In [236]:
Zy_train.value_counts()


0    79812
1    79812
Name: is_attributed, dtype: int64

In [237]:
import xgboost as xgb

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

xgb_model = xgb.XGBClassifier()
xgb_model.fit(ZX_train,Zy_train)
y_pred = xgb_model.predict(ZX_test)
print(accuracy_score(y_pred, Zy_test))




0.98065


In [238]:
print(classification_report(y_pred, Zy_test))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     19582
           1       0.90      0.08      0.15       418

    accuracy                           0.98     20000
   macro avg       0.94      0.54      0.57     20000
weighted avg       0.98      0.98      0.97     20000



In [239]:
from sklearn.metrics import confusion_matrix,roc_auc_score

%matplotlib inline
pd.DataFrame(confusion_matrix(Zy_test,y_pred), columns =  ['Predicted downloaded',' Predicted as not downloaded'], index = ['Actual downloaded',' Actual Not downloaded'])

,Predicted downloaded,Predicted as not downloaded
Actual downloaded,19578,383
Actual Not downloaded,4,35


In [240]:
roc_auc_score(Zy_test, y_pred)

0.9391242409878751

In [241]:
from sklearn.ensemble import BaggingClassifier

bgcl = BaggingClassifier(n_estimators=10, max_samples= .7, bootstrap=True)
bgcl = bgcl.fit(ZX_train, Zy_train)
test_pred = bgcl.predict(X_test)
bgcl_score=bgcl.score(ZX_test, Zy_test)
print("The Bagging classifier model accuracy is {}".format(bgcl_score))
print("The confusion matrix of Bagging classifier model is")
print(metrics.confusion_matrix(Zy_test,test_pred))

The Bagging classifier model accuracy is 0.99645
The confusion matrix of Bagging classifier model is
[[19961     0]
 [   39     0]]


In [243]:
roc_auc_score(Zy_test, test_pred)

0.5